# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [12]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [13]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

- **VMC** : vecino mas cercano
- **AGM** : arbol generador minimo
- **I** : incersion
- **tabu falta** : tabu

In [14]:
def correr_experimento(metodo, archivo_instancia,parametros_tabu):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    cmds = ["../tp2", metodo] + parametros_tabu
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(cmds , stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(f"Hubo un error en la experimentacion para el algoritmo: {metodo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo = process.stdout.read() 
    #print(process.stdout.read()) # resultado del metodo
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    costo = costo.split(" ")[1]
    costo = costo.split("\n")[0]
    res = (tiempo_de_ejecucion,float(costo))
    
    return res;




## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [20]:
experimentos = [];
experimentos

[]

## ch150

In [16]:
VMC_TS1 = [100,500,50,50]

VMC_TS2 = [100,500,300,50]

AGM_TS1 = [200,500,50,50]

AGM_TS2 = [100,500,300,100]

I_TS1 = [100,500,100,100] 

I_TS2 = [50,500,50,100]


In [17]:
def strto (s):
    jorge = []
    for a in s:
        jorge.append(str(a))
    return jorge

In [9]:
strto(AGM_TS1)

['200', '500', '50', '50']

In [21]:
data = "ch150"
n = 150
archivo_ch = "instancias_salida/ch150.txt"

experimentos.append([data, n ,"AGM", archivo_ch,[str(0),str(0),str(0),str(0)]  ]);
experimentos.append([data, n ,"AGM-TS1", archivo_ch,strto(AGM_TS1)  ]);
experimentos.append([data, n ,"AGM-TS2", archivo_ch, strto(AGM_TS2)   ]);

experimentos.append([data, n ,"VMC", archivo_ch,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"VMC-TS1", archivo_ch, strto(VMC_TS1)]    );
experimentos.append([data, n ,"VMC-TS2", archivo_ch,  strto(VMC_TS2)   ]);

experimentos.append([data, n ,"I", archivo_ch,[str(0),str(0),str(0),str(0)]]);
experimentos.append([data, n ,"I-TS1", archivo_ch,strto(I_TS1)   ]);
experimentos.append([data, n ,"I-TS2", archivo_ch, strto(I_TS2)   ]);

## CORRER EXPERIMENTOS

In [22]:
columnas = ["dataset", "n", "metodo", "tiempo","costo","largolista","max_iter","rang_iter","percentage",];


filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    costos = []
    resultados = []
    for i in range(0, T):
        exp = correr_experimento(experimento[2], experimento[3],experimento[4])
        t = exp[0]
        c = exp[1]
        costos.append(c)
        tiempos.append(t);
    costo = np.median(costos)
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo,costo,
                  experimento[4][0],experimento[4][1],experimento[4][2],experimento[4][3]]);
print('fin :)')

'Experimento: 9/9'

fin :)


In [23]:
df_resultados = pd.DataFrame(filas, columns=columnas);

#df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [24]:
df_resultados

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,ch150,150,AGM,1.472310,9315.0,0,0,0,0
1,ch150,150,AGM-TS1,5390.930000,8587.0,200,500,50,50
2,ch150,150,AGM-TS2,10251.000000,7507.0,100,500,300,100
3,ch150,150,VMC,0.685514,8191.0,0,0,0,0
4,ch150,150,VMC-TS1,5263.050000,8098.0,100,500,50,50
5,ch150,150,VMC-TS2,5220.460000,8085.0,100,500,300,50
6,ch150,150,I,1043.420000,13855.0,0,0,0,0
7,ch150,150,I-TS1,10775.000000,7876.0,100,500,100,100
8,ch150,150,I-TS2,11399.800000,7737.0,50,500,50,100


In [25]:
df_resultados.to_csv("expComparacion.csv", index=False, header=True);

In [91]:
df_resultados

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,berlin52,52,AGM-TS1,110.460,10402.0,50,1000,50,5
1,berlin52,52,AGM-TS1,112.365,10402.0,100,1000,50,5
2,berlin52,52,AGM-TS1,123.477,10402.0,200,1000,50,5
3,berlin52,52,AGM-TS1,356.553,9468.0,50,1000,50,30
4,berlin52,52,AGM-TS1,345.430,9493.0,100,1000,50,30
...,...,...,...,...,...,...,...,...,...
211,berlin52,52,I-TS2,567.208,8370.0,100,1000,300,50
212,berlin52,52,I-TS2,572.984,8497.0,200,1000,300,50
213,berlin52,52,I-TS2,1016.900,7866.0,50,1000,300,100
214,berlin52,52,I-TS2,1023.090,7887.0,100,1000,300,100


In [92]:
df_viejo = pd.read_csv("resultados/resultado.csv")

In [93]:
df_viejo

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,berlin52,52,AGM-TS1,5.750460,10402.0,50,50,50,5
1,berlin52,52,AGM-TS1,5.450580,10402.0,100,50,50,5
2,berlin52,52,AGM-TS1,5.540750,10402.0,200,50,50,5
3,berlin52,52,AGM-TS1,15.778300,9896.0,50,50,50,30
4,berlin52,52,AGM-TS1,15.721000,9896.0,100,50,50,30
...,...,...,...,...,...,...,...,...,...
646,berlin52,52,I-TS2,498.997000,8052.0,100,500,300,100
647,berlin52,52,I-TS2,493.535000,8067.0,200,500,300,100
648,berlin52,52,AGM,0.122992,10402.0,0,0,0,0
649,berlin52,52,VMC,0.077672,8980.0,0,0,0,0


In [94]:
df_nuevo = df_viejo.append(df_resultados, ignore_index = True)
df_nuevo

,dataset,n,metodo,tiempo,costo,largolista,max_iter,rang_iter,percentage
0,berlin52,52,AGM-TS1,5.75046,10402.0,50,50,50,5
1,berlin52,52,AGM-TS1,5.45058,10402.0,100,50,50,5
2,berlin52,52,AGM-TS1,5.54075,10402.0,200,50,50,5
3,berlin52,52,AGM-TS1,15.77830,9896.0,50,50,50,30
4,berlin52,52,AGM-TS1,15.72100,9896.0,100,50,50,30
...,...,...,...,...,...,...,...,...,...
862,berlin52,52,I-TS2,567.20800,8370.0,100,1000,300,50
863,berlin52,52,I-TS2,572.98400,8497.0,200,1000,300,50
864,berlin52,52,I-TS2,1016.90000,7866.0,50,1000,300,100
865,berlin52,52,I-TS2,1023.09000,7887.0,100,1000,300,100


In [95]:
df_nuevo.to_csv("resultados/resultado.csv", index=False, header=True);